In [1]:
import pandas as pd
import openai
import os
from pathlib import Path
import json
import os
import re
from collections import Counter

## GPT API Calls

In [2]:
## Paste your API key here
AA = ""
openai.api_key = AA

In [3]:
def extract_list_items(text):
    # Find all list items using regex, including those on the same line
    list_items = re.findall(r'\d+\.\s*(.+?)(?=\s*\d+\.|\n|$)', text, flags=re.DOTALL)

    # Remove the numbers from each list item
    items_without_numbers = [re.sub(r'^\s*\d+\.\s*', '', item) for item in list_items]

    # Return the extracted list items as a single string, joined by newlines
    return '\n'.join(items_without_numbers)

In [4]:
def gpt3_response(messages, temperature=0, max_tokens=1000, top_p=1, frequency_penalty=0, presence_penalty=0, best_of=1):
    """
    Func to make OpenAI API calls.
    """
    model = "gpt-3.5-turbo"
    response = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    temperature=temperature,
    max_tokens=max_tokens,
    top_p=top_p,
    frequency_penalty=frequency_penalty,
    presence_penalty=presence_penalty
    )['choices'][0]['message']['content']
    return response

In [5]:
def create_synthetic_data(input_path: str, output_path: str):
    # Load input prompts from JSON file
    with open(input_path, encoding='utf8') as j:
        prompts = json.load(j)
    
    # Generate synthetic data for each prompt using GPT-3
    data_list = [{
        'prompt_id': prompt['prompt_id'],
        'label': prompt['prompt_id'].split('_')[1].split('-')[0],
        'adverse': prompt['prompt_id'].split('_')[1].split('-')[1],
        'writer': input_path.split('/')[-1].split('.')[0],
        'synthetic_data': gpt3_response(messages=prompt['prompt'], temperature=0.3, frequency_penalty=1.9, presence_penalty=0.9, max_tokens=4000) # Increase temperature and max tokens for longer and more varied output
        } for prompt in prompts]
    data = pd.DataFrame(data_list)
    
    # If the output file already exists, append the new data to it
    if os.path.isfile(output_path):
        df = pd.read_csv(output_path)
        df = pd.concat([df, data], ignore_index=True)
    else:
        df = pd.DataFrame(data)
    
    # Extract list items from the GPT-3 response and store in a new column
    df['formatted_response'] = df['synthetic_data'].apply(extract_list_items)
    
    # Write the DataFrame to the output file
    df.to_csv(output_path, index=False)

In [ ]:
input_path = ''
output_path = ''
create_synthetic_data(input_path, output_path)